# Predict rain(precipitation) based on other weather variables

This notebook will use time lags to train a machine learning model for predicting rain (precipitation). 

First, we select a random station. The data is kept at daily resolution. Then, we generate a lagged feature matrix.

In [1]:
import pandas as pd
import numpy as np
from numpy.random import randint
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import time

In [2]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [3]:
%%time
data_dir = '/datasets/NOAA_SST/'
#load(data_dir + “noaa_gsod/…/file”
t0 = time.time()
data = pd.read_pickle(data_dir+'noaa_gsod/Combined_noaa_gsod') # load weather data
stations = pd.read_pickle(data_dir+'noaa_gsod.stations') # load station data

# # USE ONLY 2008-2018 # #
data = data.loc[data.index >= pd.Timestamp(2008, 1, 1)]
data = data.drop(columns=['yr','year','da','mo']) # don't need these anymore
print(time.time()-t0)

23.34427833557129
CPU times: user 13.9 s, sys: 9.5 s, total: 23.4 s
Wall time: 23.3 s


In [4]:
# # SELECT RANDOM STATION # #
np.random.seed(30)
rs = np.unique(data['stn'].values) # find unique stations with data
rand_stat = rs[randint(len(rs))] # pick a random station

# # ideally we should check < len(np.unique(data.index)), but many are shorter
while (len(data.loc[data['stn'] == rand_stat]) < 3650): # If not enough data
    if len(stations.loc[stations['usaf'] == rand_stat]): # If station info available
        if (stations.loc[stations['usaf'] == rand_stat].iloc[0]['wban'] != '99999'): # If station number not unique
            rand_stat = rs[randint(len(rs))] # get a new station
    else:
        rand_stat = rs[randint(len(rs))] # get a new station

select_station = stations.loc[stations['usaf'] == rand_stat] # get location, etc, for random station

features = data.loc[data['stn'] == rand_stat] # pick weather at random station
features = features.drop(columns='stn')
features = features.sort_index()
select_station.head() # see where it is

,usaf,wban,name,country,state,call,lat,lon,elev,begin,end
24140,711320,99999,STONY RAPIDS ARPT,CA,None,CYSF,59.25,-105.833,+0250.0,19860725,20190401


In [5]:
features.shape

(3994, 7)

### Time-shift the data

In [6]:
columns = features.columns # weather variables
for co in columns:
    # one day lag
    features[co + '_lag1'] = features[co].shift(periods=1)
    
    # two days lag
    features[co + '_lag2'] = features[co].shift(periods=2)
    
    # three days lag
    features[co + '_lag3'] = features[co].shift(periods=3)
features = features.iloc[3:]
print(str(len(features)) + ' samples, ' + str(len(features.columns)) + ' features.')
features.head()

3991 samples, 28 features.


,temp,slp,wdsp,mxpsd,max,min,prcp,temp_lag1,temp_lag2,temp_lag3,...,mxpsd_lag3,max_lag1,max_lag2,max_lag3,min_lag1,min_lag2,min_lag3,prcp_lag1,prcp_lag2,prcp_lag3
Datetime,,,,,,,,,,,,,,,,,,,,,
2008-01-04,7.9,998.0,3.4,9.9,19.4,-4.0,0.00,9.2,-6.9,-8.3,...,5.1,14.0,8.6,-2.2,-0.4,-20.2,-16.6,0.04,0.00,0.01
2008-01-05,6.8,995.4,9.7,12.0,18.9,-4.9,0.13,7.9,9.2,-6.9,...,9.9,19.4,14.0,8.6,-4.0,-0.4,-20.2,0.00,0.04,0.00
2008-01-06,4.0,993.5,8.5,13.0,9.7,2.7,0.09,6.8,7.9,9.2,...,9.9,18.9,19.4,14.0,-4.9,-4.0,-0.4,0.13,0.00,0.04
2008-01-07,4.7,1001.9,0.6,2.9,5.5,3.0,0.05,4.0,6.8,7.9,...,9.9,9.7,18.9,19.4,2.7,-4.9,-4.0,0.09,0.13,0.00
2008-01-08,-4.2,1011.6,0.3,2.9,5.5,-8.3,0.06,4.7,4.0,6.8,...,12.0,5.5,9.7,18.9,3.0,2.7,-4.9,0.05,0.09,0.13


### Create train/val/test

In [7]:
ylabel = features['prcp'] # use today's precipitation as the label
features = features.drop(columns='prcp') # don't put it in training data!!

# Use 20% test split (80% training + validation)
ntrain = int(len(features)*0.8)
x_test = features.iloc[ntrain:,:]
y_test = ylabel[ntrain:]
indices = np.arange(ntrain)


# Split remaining 80% into training-validation sets by 60%/20% (of original data)
x_train, x_val, y_train, y_val, idx1, idx2 = train_test_split(features.iloc[0:ntrain,:], ylabel[0:ntrain], \
                                                   indices, test_size=0.2, random_state=1)

# # Scale features. Fit scaler on training only.
scaler = MinMaxScaler() #scale features between 0 and 1
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

### Predict with Random Forest

In [8]:
int(len(features)*.8)

3140

In [8]:
# # Create, train, and predict random forest here # #

clf = RandomForestRegressor(n_estimators=100, random_state = 10)
clf.fit(x_train, y_train)

# # Also predict on the training and validation data
y_train_pred = clf.predict(x_train)
y_val_pred = clf.predict(x_val)

y_test_pred = clf.predict(x_test)



In [9]:
from sklearn.metrics import mean_squared_error

In [10]:
print("test_accuracy: {0}".format(mean_squared_error(y_test, y_test_pred)))
print("train_accuracy: {0}".format(mean_squared_error(y_train, y_train_pred)))
print("val_accuracy: {0}".format(mean_squared_error(y_val, y_val_pred)))

test_accuracy: 0.01178678555694618
train_accuracy: 0.0013904420485703092
val_accuracy: 0.008949142300469485


In [ ]:
train_pred_df = pd.Series(data = y_train_pred, index = features.iloc[idx1].index)
val_pred_df = pd.Series(data = y_val_pred, index = features.iloc[idx2].index)

### Plot random forest

In [ ]:
regr = clf

In [ ]:
# plot predictions
fig, ax = plt.subplots(1,1,figsize=(10,8))
ax.plot(features.iloc[ntrain:].index,y_test, 'r', label="Actual Test Temperature") # plot actual temperature
ax.plot(features.iloc[ntrain:].index, y, 'b', label="Predicted Test Temperature") # plot predicted temperature

# # PLOT THE PREDICTED TRAINING AND VALIDATION DATA HERE # #
ax.plot(features.iloc[0:ntrain].index, ylabel[0:ntrain], 'y', label="Actual Train Temperature")
ax.plot(features.iloc[0:ntrain].index, train_pred_df, 'k', label="Predicted Train Temperature")

#plt.plot(features.iloc[val_ind].index, y_val, 'm', label="Actual Val Temperature")
#plt.plot(features.iloc[val_ind].index, y_val_pred, 'c', label="Predicted Val Temperature")

# # INCREASE X TICK SPACING, UPDATE LEGEND # #
ax.set_xticks(features.iloc[:].index[::182]) # set xticks to monthly
ax.set_xticklabels(ax.get_xticks(), rotation=45)
myFmt = mdates.DateFormatter('%b-%y')
plt.gca().xaxis.set_major_formatter(myFmt)
ax.set_title('Training and Test predictions and true values')
ax.set_ylabel('Daily Temperature (degree Fahrenheit)', fontsize=12)
ax.set_xlabel('Date(Month-Year)')
lgd = ax.legend(bbox_to_anchor=(1, 1.2), loc='upper center')
fig.savefig("problem 1.png", bbox_extra_artists=(lgd,), bbox_inches='tight')
plt.show()

# # Plot the feature importances # #


In [ ]:
nfeatures = 10
fi = regr.feature_importances_ # get feature importances, regr is the regressor
fi_sort = np.argsort(fi)[::-1] # sort importances most to least
plt.subplot(1,2,2)
plt.bar(range(nfeatures), fi[fi_sort[0:nfeatures]], width=1, \
        tick_label=features.columns.values[fi_sort[0:nfeatures]]) # plot features importances
plt.ylabel('Feature Importance (avg across trees)', fontsize=12)
plt.xticks(rotation = 45)
plt.show()

In [ ]:
# plot predictions
plt.figure(figsize=(15,7))
plt.subplot(1,2,1)
plt.plot(features.iloc[ntrain:].index, y_test) # plot actual precipitation
plt.plot(features.iloc[ntrain:].index, y) # plot predicted precipitation, y is the prediction results on testing data


# # PLOT TRAINING DATA HERE # #





# # INCREASE X TICK SPACING, UPDATE LEGEND # #
plt.xticks(features.index[::182], rotation = 45) # X-Ticks are spaced once every 30 days. 
myFmt = mdates.DateFormatter('%b-%y') # This shows day-month-year. Switch to month-year or annually

plt.gca().xaxis.set_major_formatter(myFmt)
plt.ylabel('Daily Precipitation (inches)', fontsize=12)
plt.legend(('Test Label','Test Prediction','Train Label','Train Prediction'), fontsize=12, loc=1)
#plt.show()

# # Plot the feature importances # #
nfeatures = 10
fi = regr.feature_importances_ # get feature importances, regr is the regressor
fi_sort = np.argsort(fi)[::-1] # sort importances most to least
plt.subplot(1,2,2)
plt.bar(range(nfeatures), fi[fi_sort[0:nfeatures]], width=1, \
        tick_label=features.columns.values[fi_sort[0:nfeatures]]) # plot features importances
plt.ylabel('Feature Importance (avg across trees)', fontsize=12)
plt.xticks(rotation = 45)
plt.show()

### Feature importance is the weighted impurity of a branch adjusted by its children nodes and normalized by the impurities of all branches. The Random Forest feature importances are averaged over all regression trees.